In [1]:
#define your parameters here
#used reference from here: https://www.kaggle.com/code/prakharbatwal/transformer
pretrained_model = 'bert-base-uncased'
num_epochs = 3
learning_rate = 5e-5
batch_size = 8

In [2]:
import torch

if(torch.cuda.is_available()):
    device = "cuda"
else:
    device = "cpu"

print(f"using {device} for processing")

using cpu for processing


In [3]:
import pandas as pd
#dataset ref: https://opus.nlpl.eu/WikiMatrix/en&ko/v1/WikiMatrix#download

# Define file paths (adjust if your path is different)
en_path = "en-ko.txt/WikiMatrix.en-ko.en"   # English sentences
ko_path = "en-ko.txt/WikiMatrix.en-ko.ko"   # Korean sentences

# Read files line by line
with open(en_path, encoding='utf-8') as f_en, open(ko_path, encoding='utf-8') as f_ko:
    en_lines = f_en.readlines()
    ko_lines = f_ko.readlines()

# Ensure both files have the same number of lines
assert len(en_lines) == len(ko_lines), "Mismatch in line count!"

# Combine into a pandas DataFrame
df = pd.DataFrame({
    'en': [line.strip() for line in en_lines],
    'ko': [line.strip() for line in ko_lines]
})

# Optional: drop empty rows and shuffle
df = df[df['en'].str.strip() != ""]
df = df[df['ko'].str.strip() != ""]
df.dropna(inplace=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Preview the data
print(df.head())

# Optional: Save to CSV
df.to_csv("translation_en_ko.csv", index=False)



                                                  en  \
0  Scottish soldiers also served in the Garde Éco...   
1  Under his leadership, he helped foster busines...   
2  In 1702, English colonel James Moore and allie...   
3  In the middle of the 19th century, it was rest...   
4                Other research facilities include .   

                                                  ko  
0   또한 스코틀랜드 병사들은 프랑스의 국왕의 친위 부대 가르드 에코세즈에 복무하기도 했다.  
1                  조선업을 중심으로 산업을 육성하여 일본의 근대화에 기여했다.  
2  1702년 영국의 제임스 무어 대령과 동맹한 야마시 족과 크릭 족이 세인트 오거스틴...  
3                                 19세기에 한 차례 더 수리했다.  
4                                이와는 다른 연구 시각도 존재한다.  


In [4]:
import numpy as np

from sklearn.utils import shuffle
df = shuffle(df, random_state=42)

In [5]:
from sklearn.model_selection import train_test_split
X = df['ko'].tolist()
y = df['en'].tolist()
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 42)

In [6]:
type(y_train)

list

In [7]:
print("Train Label Counts:", pd.Series(y_train).value_counts())


Train Label Counts: "1921 Age and beauty".                                                                                                                                                                                                                                   1
Bluetooth beacons can be used to send a packet of information that contains a Universally Unique Identifier (UUID).                                                                                                                                      1
Despite his narcissism and selfishness, Drake still redeems himself by doing the right thing.                                                                                                                                                            1
Like Shelley, Starbuck was a skilled singles skater.                                                                                                                                                                               

In [8]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

train_encodings = tokenizer(X_train, truncation=True, padding=True, max_length=256,return_tensors="pt").to(device)
test_encodings = tokenizer(X_test, truncation=True, padding=True, max_length=256,return_tensors="pt").to(device)

C:\Users\Amrita Aryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
from torch.utils.data import Dataset
import torch

class TranslationDataset(Dataset):
    def __init__(self, inputs, targets):
        self.inputs = inputs
        self.targets = targets

    def __len__(self):
        return len(self.inputs["input_ids"])

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.inputs.items()}
        item["labels"] = self.targets["input_ids"][idx]
        return {k: torch.tensor(v) for k, v in item.items()}


In [10]:
# Make sure X_train and y_train are lists of strings
train_inputs = tokenizer(X_train, padding=True, truncation=True, max_length=256, return_tensors="pt")
train_labels = tokenizer(y_train, padding=True, truncation=True, max_length=256, return_tensors="pt")

test_inputs = tokenizer(X_test, padding=True, truncation=True, max_length=256, return_tensors="pt")
test_labels = tokenizer(y_test, padding=True, truncation=True, max_length=256, return_tensors="pt")


In [11]:
train_dataset = TranslationDataset(train_inputs, train_labels)
test_dataset = TranslationDataset(test_inputs, test_labels)

print(train_dataset[0])  # Should show input_ids, attention_mask, and labels


{'input_ids': tensor([  101,  2355, 29992, 30010, 30021,  1021, 29999, 30015, 30022,  1015,
        29999, 30019, 30022, 29996, 30014, 30003, 30008,  1010, 16914,  2696,
         1463, 30014, 30021, 29999, 30019, 30023, 29999, 30017, 30022,  1467,
        30011, 30025, 30005, 30007, 30025,  1459, 30011, 30020, 30000, 30008,
        30020, 29999, 30009,   100,  1457, 30006, 30022, 29994, 30019,  1461,
        30008, 30022, 30000, 30008, 30025, 30005, 30007,   100,  1012,   102,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 

C:\Users\Amrita Aryan\AppData\Local\Temp\ipykernel_32080\1327556567.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {k: torch.tensor(v) for k, v in item.items()}


In [12]:
for i in range(5):
    sample = train_dataset[i]
    print(f"Label: {sample['labels']}, Decoded: {tokenizer.decode(sample['input_ids'])[:100]}")


Label: tensor([  101,  2004,  1997,  2251,  1015,  1010,  2355,  1010, 16914,  2696,
        27092,  2024,  2241,  2006,  1996,  4440,  2828,  1012,   102,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0, 

C:\Users\Amrita Aryan\AppData\Local\Temp\ipykernel_32080\1327556567.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {k: torch.tensor(v) for k, v in item.items()}


In [13]:
print(type(train_encodings))
print(train_encodings.keys())


<class 'transformers.tokenization_utils_base.BatchEncoding'>
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])


In [14]:
sample = train_dataset.__getitem__(1)
print("Keys:", sample.keys())
print("Input IDs:", sample["input_ids"][:10])  # Show first 10 tokens
print("Label:", sample["labels"])
print("Decoded:", tokenizer.decode(sample["input_ids"]))


Keys: dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])
Input IDs: tensor([  101,  6068, 29992, 30010, 30021,  2340, 29999, 30015, 30022,  1023])
Label: tensor([  101,  2006,  1023,  2281,  6068,  1010,  1996,  2586,  4078, 18341,
         2015, 18427,  2015,  2139,  2998,  2012,  7317, 20624, 10997,  1006,
         1000,  2586,  1997,  1996,  2413,  8384,  2005,  5188,  2998,  1000,
         2030,  2149, 10343,  2050,  1007,  2404,  2041,  2019,  8874,  2008,
         2009,  2052,  2031,  7082,  2157,  2000,  2151,  7362,  4368,  2218,
         2076,  1996,  2088,  1005,  1055,  4189,  1012,   102,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,    

C:\Users\Amrita Aryan\AppData\Local\Temp\ipykernel_32080\1327556567.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {k: torch.tensor(v) for k, v in item.items()}


In [15]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, num_labels=2).to(device)

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    save_strategy="epoch", #saves the model after every epoch into the out directory
    logging_dir="./logs",
    logging_steps=10,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    learning_rate = learning_rate,
    gradient_accumulation_steps=1,
    fp16=True,
    report_to="none",
    dataloader_pin_memory=True if device == "cpu" else False
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)
print("Train size:", len(train_dataset))
print("Test size:", len(test_dataset))

# Sanity check random entries
print("Sample train label:", y_train[0])
print("Sample test label:", y_test[0])
print("Sample train text:", X_train[0][:300])  # preview first 300 chars
print("Sample test text:", X_test[0][:300])



Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train size: 245520
Test size: 61380
Sample train label: As of July 1, 2016, MBTA fares are based on the trip type.
Sample test label: Bolan's death ended the band.
Sample train text: 2016년 7월 1일부터, MBTA 운임을 통행 목적에 따라 달리 설정해 놓았다.
Sample test text: 볼란의 죽음으로 밴드는 끝이 났다.


In [16]:
model.device

device(type='cpu')

In [17]:
trainer.train()
trainer.evaluate()

C:\Users\Amrita Aryan\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
C:\Users\Amrita Aryan\AppData\Local\Temp\ipykernel_32080\1327556567.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return {k: torch.tensor(v) for k, v in item.items()}


ValueError: Expected input batch_size (8) to match target batch_size (2048).

In [ ]:
from sklearn.metrics import classification_report
import numpy as np
from sklearn.metrics import confusion_matrix


preds = trainer.predict(test_dataset)
print("Raw logits for first 5 samples:\n", preds.predictions[:5])

y_pred = np.argmax(preds.predictions, axis=1)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


/tmp/ipykernel_35/2811690718.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}


Raw logits for first 5 samples:
 [[-6.0351562  6.6992188]
 [ 6.25      -6.8789062]
 [ 6.2460938 -6.859375 ]
 [ 6.2226562 -6.9101562]
 [-6.0351562  6.703125 ]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3521
           1       1.00      1.00      1.00      4208

    accuracy                           1.00      7729
   macro avg       1.00      1.00      1.00      7729
weighted avg       1.00      1.00      1.00      7729

[[3521    0]
 [   1 4207]]


In [ ]:
import torch

random_texts = ["this is gibberish", "you won a lottery", "election results"]
inputs = tokenizer(random_texts, return_tensors="pt", truncation=True, padding=True)

# Move inputs to model's device (usually cuda:0)
device = model.device
inputs = {k: v.to(device) for k, v in inputs.items()}

# Predict
outputs = model(**inputs)
preds = torch.argmax(outputs.logits, axis=1)
print(preds)


tensor([0, 0, 0], device='cuda:0')
